This is the version of code that has the best performance.
The general idea is to employ Scibert and fine-tune afterwards. Dataset was divided into five folds and earlystopping is applied.
Final prediction is from ensembled method, which corresponds to the  label with highest average probability. 

In [ ]:
!pip install transformers

     |████████████████████████████████| 2.0MB 7.9MB/s 
     |████████████████████████████████| 3.2MB 22.1MB/s 
     |████████████████████████████████| 890kB 50.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=bf03485d1bab763ee2ed8b23a6b00812f1057bb1ebf53d769f629f23bed95bae
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
import pandas as pd
import numpy as np
import warnings

import torch
from transformers import BertTokenizer, BertModel, BertForSequenceClassification
from tqdm import tqdm_notebook
from sklearn.metrics import f1_score, accuracy_score
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, BertConfig, get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, random_split, DataLoader, RandomSampler, SequentialSampler
import keras
import time
import datetime
import time
from sklearn.model_selection import StratifiedKFold
import random
import os
warnings.filterwarnings('ignore')


KFOLDS = 5
BATCH_SIZE = 32
SEED = 57
EPOCHS = 8
EARLYSTOP_NUM = 2
device = torch.device('cuda')

In [ ]:
class EarlyStopping:
    def __init__(self, patience=7, mode='max', delta=0.001):
        self.patience = patience
        self.counter = 0
        self.mode = mode
        self.best_score = None
        self.early_stop = False
        self.delta = delta
        if self.mode == 'min':
            self.val_score = np.Inf
        else:
            self.val_score = -np.Inf
            
    def __call__(self, epoch_score, model, model_path):
        
        if self.mode == 'min':
            score = -1.0 * epoch_score
        else:
            score = np.copy(epoch_score)
            
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(epoch_score, model, model_path)
        elif score < self.best_score:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(epoch_score, model, model_path)
            self.counter = 0    
        
    def save_checkpoint(self, epoch_score, model, model_path):
        if epoch_score not in [-np.inf, np.inf, -np.nan, np.nan]:
            torch.save(model.state_dict(), model_path)
        self.val_score = epoch_score
        

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
    
def flat_accuracy(preds, labels):
    
    """A function for calculating accuracy scores"""
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    
    return accuracy_score(labels_flat, pred_flat)

def format_time(elapsed):    
    
    """A function that takes a time in seconds and returns a string hh:mm:ss"""
    
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
train = pd.read_csv('/content/gdrive/My Drive/train.csv', names=['label', 'node idx'])
text  = pd.read_csv('/content/gdrive/My Drive/text.csv', names=['paper id', 'title', 'abstract'])
test  = pd.read_csv('/content/gdrive/My Drive/test.csv', names=['node idx'])
node2paper = pd.read_csv('/content/gdrive/My Drive/nodeid2paperid.csv', header=0)
sample = pd.read_csv('/content/gdrive/My Drive/sample.csv')


train = pd.merge(train, node2paper, on='node idx')
train = pd.merge(train, text, on='paper id')
test = pd.merge(test, node2paper, on='node idx')
test = pd.merge(test, text, on='paper id')

In [ ]:
!wget -O scibert_uncased.tar https://s3-us-west-2.amazonaws.com/ai2-s2-research/scibert/huggingface_pytorch/scibert_scivocab_uncased.tar
!tar -xvf scibert_uncased.tar

--2021-03-23 14:05:40--  https://s3-us-west-2.amazonaws.com/ai2-s2-research/scibert/huggingface_pytorch/scibert_scivocab_uncased.tar
Resolving s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)... 52.218.249.200
Connecting to s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)|52.218.249.200|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 442460160 (422M) [application/x-tar]
Saving to: ‘scibert_uncased.tar’

scibert_uncased.tar 100%[===================>] 421.96M  19.3MB/s    in 23s     

2021-03-23 14:06:05 (18.0 MB/s) - ‘scibert_uncased.tar’ saved [442460160/442460160]

scibert_scivocab_uncased/
scibert_scivocab_uncased/vocab.txt
scibert_scivocab_uncased/pytorch_model.bin
scibert_scivocab_uncased/config.json


In [ ]:
model_version = 'scibert_scivocab_uncased'
do_lower_case = True
model = BertForSequenceClassification.from_pretrained(model_version,
                                                      num_labels=20,
                                                      output_attentions = False, # Whether the model returns attentions weights.
                                                      output_hidden_states = False, # Whether the model returns all hidden-states.
                                                     )
tokenizer = BertTokenizer.from_pretrained(model_version, do_lower_case=do_lower_case)

Some weights of the model checkpoint at scibert_scivocab_uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkp

In [ ]:
def tokenize_map(sentence,labs='None'):
    
    """A function for tokenize all of the sentences and map the tokens to their word IDs."""
    
    global labels
    
    input_ids = []
    attention_masks = []

    # For every sentence...
    
    for text in tqdm_notebook(sentence):
        #   "encode_plus" will:
        
        #   (1) Tokenize the sentence.
        #   (2) Prepend the `[CLS]` token to the start.
        #   (3) Append the `[SEP]` token to the end.
        #   (4) Map tokens to their IDs.
        #   (5) Pad or truncate the sentence to `max_length`
        #   (6) Create attention masks for [PAD] tokens.
        
        encoded_dict = tokenizer.encode_plus(
                            text,                      # Sentence to encode.
                            add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                            truncation='longest_first', # Activate and control truncation
                            max_length = 256,           # Max length according to our text data.
                            pad_to_max_length = True, # Pad & truncate all sentences.
                            return_attention_mask = True,   # Construct attn. masks.
                            return_tensors = 'pt',     # Return pytorch tensors.
                       )

        # Add the encoded sentence to the id list. 
        
        input_ids.append(encoded_dict['input_ids'])

        # And its attention mask (simply differentiates padding from non-padding).
        
        attention_masks.append(encoded_dict['attention_mask'])

    # Convert the lists into tensors.
    
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    
    if labs != 'None': # Setting this for using this definition for both train and test data so labels won't be a problem in our outputs.
        labels = torch.tensor(labs)
        return input_ids, attention_masks, labels
    else:
        return input_ids, attention_masks

In [ ]:
train['text'] = train['title'] + '. ' + train['abstract']

In [ ]:
skf = StratifiedKFold(n_splits=KFOLDS, shuffle=True, random_state=SEED)
seed_everything(SEED)
total_t0 = time.time()
total_training_stats = []
oof = np.zeros((len(train), 20))

for fold, (train_idx, valid_idx) in enumerate(skf.split(train, train.label.values)):
    print(f'Fold: {fold} training...')
    
    #-------------------prepare_data---------------------------#
    kfold_train = train.loc[train_idx]
    kfold_valid = train.loc[valid_idx]
    
    #tr_input_ids, tr_attention_masks, tr_labels = tokenize_map(kfold_train.abstract.values, kfold_train.label.values)
    tr_input_ids, tr_attention_masks, tr_labels = tokenize_map(kfold_train.text.values, kfold_train.label.values)
    traindataset = TensorDataset(tr_input_ids, tr_attention_masks, tr_labels)
    
    #val_input_ids, val_attention_masks, val_labels = tokenize_map(kfold_valid.abstract.values, kfold_valid.label.values)
    val_input_ids, val_attention_masks, val_labels = tokenize_map(kfold_valid.text.values, kfold_valid.label.values)
    validdataset = TensorDataset(val_input_ids, val_attention_masks, val_labels)
    
    train_dataloader = DataLoader(
                                  traindataset,  # The training samples.
                                  sampler = RandomSampler(traindataset), # Select batches randomly
                                  batch_size = BATCH_SIZE # Trains with this batch size.
                                 )    
    
    validation_dataloader = DataLoader(
                                        validdataset, # The validation samples.
                                        sampler = SequentialSampler(validdataset), # Pull out batches sequentially.
                                        batch_size = BATCH_SIZE # Evaluate with this batch size.
                                       ) 
    
    
    #-------------------get_model---------------------------#
    torch.cuda.empty_cache()
    model = BertForSequenceClassification.from_pretrained(model_version,
                                                          num_labels=20,
                                                          output_attentions = False, # Whether the model returns attentions weights.
                                                          output_hidden_states = False, # Whether the model returns all hidden-states.
                                                         )
    model.to(device)
    optimizer = AdamW(model.parameters(), lr = 2e-5, eps = 1e-7, weight_decay = 1e-6)
    total_steps = len(train_dataloader) * EPOCHS
    
    scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)
    
    es = EarlyStopping(patience=EARLYSTOP_NUM, mode='max')
    model_weights = f"model_fold_text_v2{fold}.pth"
    training_stats = []
     
    
    #-------------------train_model---------------------------#
    for epoch_i in range(0, EPOCHS):
        print('')
        print(f'fold : {fold}')
        print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, EPOCHS))
        print('Training...')

        t0 = time.time()
        total_train_loss = 0
        model.train()

        # For each batch of training data...
        for step, batch in enumerate(train_dataloader):

            # Progress update every 50 batches.
            if step % 50 == 0 and not step == 0:
                elapsed = format_time(time.time() - t0)
                print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

            
            b_input_ids = batch[0].to(device).to(torch.int64)
            b_input_mask = batch[1].to(device).to(torch.int64)
            b_labels = batch[2].to(device).to(torch.int64)

            model.zero_grad()        
            
            train_output = model(b_input_ids, 
                           token_type_ids=None, 
                           attention_mask=b_input_mask, 
                           labels=b_labels)

            loss = train_output[0]
            logits = train_output[1]
            
            total_train_loss += loss.item()

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()


        avg_train_loss = total_train_loss / len(train_dataloader)            
        training_time = format_time(time.time() - t0)

        print('')
        print('  Average training loss: {0:.4f}'.format(avg_train_loss))
        print('  Training epcoh took: {:}'.format(training_time))


        #-------------------Running_Validation---------------------------#
        print('')
        print('Running Validation...')

        t0 = time.time()
        
        model.eval()
        total_eval_accuracy = 0
        total_eval_loss = 0
        nb_eval_steps = 0
        oof_preds = []
        
        for batch in validation_dataloader:

            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)

            with torch.no_grad():        
                valid_output  = model(b_input_ids, 
                                       token_type_ids=None, 
                                       attention_mask=b_input_mask,
                                       labels=b_labels)

            loss = valid_output[0]
            logits = valid_output[1]

            total_eval_loss += loss.item()

            # Move logits and labels to CPU:
            logits = logits.detach().cpu().numpy()
            label_ids = b_labels.to('cpu').numpy()
            total_eval_accuracy += flat_accuracy(logits, label_ids)
           
            oof_preds.append(logits)        
        
        oof[valid_idx] = np.vstack(oof_preds)
        
        avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
        print('  Accuracy: {0:.4f}'.format(avg_val_accuracy))

        avg_val_loss = total_eval_loss / len(validation_dataloader)

        validation_time = format_time(time.time() - t0)

        print('  Validation Loss: {0:.2f}'.format(avg_val_loss))
        print('  Validation took: {:}'.format(validation_time))

        training_stats.append(
            {
                'epoch': epoch_i + 1,
                'Training Loss': avg_train_loss,
                'Valid. Loss': avg_val_loss,
                'Valid. Accur.': avg_val_accuracy,
                'Training Time': training_time,
                'Validation Time': validation_time
            }
        )
        
        es(avg_val_accuracy, model, model_path=model_weights)
        

        if es.early_stop:
            print(f"epoch_{epoch_i}, Early stopping")
            break
    total_training_stats.append(training_stats)
    
    
    print('Training complete!')

print('Total training took {:} (h:mm:ss)'.format(format_time(time.time()-total_t0)))   

Fold: 0 training...


Some weights of the model checkpoint at scibert_scivocab_uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkp


fold : 0
======== Epoch 1 / 8 ========
Training...
  Batch    50  of  1,500.    Elapsed: 0:00:22.
  Batch   100  of  1,500.    Elapsed: 0:00:44.
  Batch   150  of  1,500.    Elapsed: 0:01:06.
  Batch   200  of  1,500.    Elapsed: 0:01:28.
  Batch   250  of  1,500.    Elapsed: 0:01:50.
  Batch   300  of  1,500.    Elapsed: 0:02:12.
  Batch   350  of  1,500.    Elapsed: 0:02:34.
  Batch   400  of  1,500.    Elapsed: 0:02:56.
  Batch   450  of  1,500.    Elapsed: 0:03:18.
  Batch   500  of  1,500.    Elapsed: 0:03:40.
  Batch   550  of  1,500.    Elapsed: 0:04:02.
  Batch   600  of  1,500.    Elapsed: 0:04:24.
  Batch   650  of  1,500.    Elapsed: 0:04:46.
  Batch   700  of  1,500.    Elapsed: 0:05:08.
  Batch   750  of  1,500.    Elapsed: 0:05:30.
  Batch   800  of  1,500.    Elapsed: 0:05:52.
  Batch   850  of  1,500.    Elapsed: 0:06:14.
  Batch   900  of  1,500.    Elapsed: 0:06:36.
  Batch   950  of  1,500.    Elapsed: 0:06:58.
  Batch 1,000  of  1,500.    Elapsed: 0:07:20.
  Batch 

Some weights of the model checkpoint at scibert_scivocab_uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkp


fold : 1
======== Epoch 1 / 8 ========
Training...
  Batch    50  of  1,500.    Elapsed: 0:00:22.
  Batch   100  of  1,500.    Elapsed: 0:00:44.
  Batch   150  of  1,500.    Elapsed: 0:01:06.
  Batch   200  of  1,500.    Elapsed: 0:01:28.
  Batch   250  of  1,500.    Elapsed: 0:01:50.
  Batch   300  of  1,500.    Elapsed: 0:02:12.
  Batch   350  of  1,500.    Elapsed: 0:02:34.
  Batch   400  of  1,500.    Elapsed: 0:02:56.
  Batch   450  of  1,500.    Elapsed: 0:03:18.
  Batch   500  of  1,500.    Elapsed: 0:03:40.
  Batch   550  of  1,500.    Elapsed: 0:04:02.
  Batch   600  of  1,500.    Elapsed: 0:04:24.
  Batch   650  of  1,500.    Elapsed: 0:04:46.
  Batch   700  of  1,500.    Elapsed: 0:05:08.
  Batch   750  of  1,500.    Elapsed: 0:05:30.
  Batch   800  of  1,500.    Elapsed: 0:05:51.
  Batch   850  of  1,500.    Elapsed: 0:06:13.
  Batch   900  of  1,500.    Elapsed: 0:06:35.
  Batch   950  of  1,500.    Elapsed: 0:06:57.
  Batch 1,000  of  1,500.    Elapsed: 0:07:19.
  Batch 

...


fold : 2
======== Epoch 1 / 8 ========
Training...
  Batch    50  of  1,500.    Elapsed: 0:00:22.
  Batch   100  of  1,500.    Elapsed: 0:00:44.
  Batch   150  of  1,500.    Elapsed: 0:01:06.
  Batch   200  of  1,500.    Elapsed: 0:01:28.
  Batch   250  of  1,500.    Elapsed: 0:01:50.
  Batch   300  of  1,500.    Elapsed: 0:02:12.
  Batch   350  of  1,500.    Elapsed: 0:02:34.
  Batch   400  of  1,500.    Elapsed: 0:02:56.
  Batch   450  of  1,500.    Elapsed: 0:03:18.
  Batch   500  of  1,500.    Elapsed: 0:03:40.
  Batch   550  of  1,500.    Elapsed: 0:04:02.
  Batch   600  of  1,500.    Elapsed: 0:04:24.
  Batch   650  of  1,500.    Elapsed: 0:04:46.
  Batch   700  of  1,500.    Elapsed: 0:05:08.
  Batch   750  of  1,500.    Elapsed: 0:05:30.
  Batch   800  of  1,500.    Elapsed: 0:05:52.
  Batch   850  of  1,500.    Elapsed: 0:06:15.
  Batch   900  of  1,500.    Elapsed: 0:06:37.
  Batch   950  of  1,500.    Elapsed: 0:06:59.
  Batch 1,000  of  1,500.    Elapsed: 0:07:21.
  Batch 

Some weights of the model checkpoint at scibert_scivocab_uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkp


fold : 3
======== Epoch 1 / 8 ========
Training...
  Batch    50  of  1,500.    Elapsed: 0:00:22.
  Batch   100  of  1,500.    Elapsed: 0:00:44.
  Batch   150  of  1,500.    Elapsed: 0:01:06.
  Batch   200  of  1,500.    Elapsed: 0:01:28.
  Batch   250  of  1,500.    Elapsed: 0:01:50.
  Batch   300  of  1,500.    Elapsed: 0:02:12.
  Batch   350  of  1,500.    Elapsed: 0:02:34.
  Batch   400  of  1,500.    Elapsed: 0:02:56.
  Batch   450  of  1,500.    Elapsed: 0:03:18.
  Batch   500  of  1,500.    Elapsed: 0:03:40.
  Batch   550  of  1,500.    Elapsed: 0:04:02.
  Batch   600  of  1,500.    Elapsed: 0:04:24.
  Batch   650  of  1,500.    Elapsed: 0:04:46.
  Batch   700  of  1,500.    Elapsed: 0:05:08.
  Batch   750  of  1,500.    Elapsed: 0:05:30.
  Batch   800  of  1,500.    Elapsed: 0:05:52.
  Batch   850  of  1,500.    Elapsed: 0:06:14.
  Batch   900  of  1,500.    Elapsed: 0:06:36.
  Batch   950  of  1,500.    Elapsed: 0:06:58.
  Batch 1,000  of  1,500.    Elapsed: 0:07:20.
  Batch 

Some weights of the model checkpoint at scibert_scivocab_uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkp


fold : 4
======== Epoch 1 / 8 ========
Training...
  Batch    50  of  1,500.    Elapsed: 0:00:22.
  Batch   100  of  1,500.    Elapsed: 0:00:44.
  Batch   150  of  1,500.    Elapsed: 0:01:06.
  Batch   200  of  1,500.    Elapsed: 0:01:28.
  Batch   250  of  1,500.    Elapsed: 0:01:50.
  Batch   300  of  1,500.    Elapsed: 0:02:12.
  Batch   350  of  1,500.    Elapsed: 0:02:34.
  Batch   400  of  1,500.    Elapsed: 0:02:56.
  Batch   450  of  1,500.    Elapsed: 0:03:18.
  Batch   500  of  1,500.    Elapsed: 0:03:40.
  Batch   550  of  1,500.    Elapsed: 0:04:02.
  Batch   600  of  1,500.    Elapsed: 0:04:24.
  Batch   650  of  1,500.    Elapsed: 0:04:46.
  Batch   700  of  1,500.    Elapsed: 0:05:08.
  Batch   750  of  1,500.    Elapsed: 0:05:30.
  Batch   800  of  1,500.    Elapsed: 0:05:52.
  Batch   850  of  1,500.    Elapsed: 0:06:14.
  Batch   900  of  1,500.    Elapsed: 0:06:36.
  Batch   950  of  1,500.    Elapsed: 0:06:58.
  Batch 1,000  of  1,500.    Elapsed: 0:07:20.
  Batch 

In [ ]:
test['text'] = test['title'] + '. ' + test['abstract']

In [ ]:
test_input_ids, test_attention_masks= tokenize_map(test.text.values)
prediction_data = TensorDataset(test_input_ids, test_attention_masks)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=BATCH_SIZE)

In [ ]:
result = np.zeros((len(test), 20))
for fold in range(KFOLDS):
    torch.cuda.empty_cache()
    model = BertForSequenceClassification.from_pretrained(model_version,
                                                          num_labels=20,
                                                          output_attentions = False, # Whether the model returns attentions weights.
                                                          output_hidden_states = False, # Whether the model returns all hidden-states.
                                                         )

    model.to(device)
    model_weights = f"model_fold_text_v2{fold}.pth"
    model.load_state_dict(torch.load(model_weights))
    model.eval()
    print('load model done!')
    print('begin inference')
    
    preds = []
    for batch in tqdm_notebook(prediction_dataloader):
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        
        with torch.no_grad():        
            test_output = model(b_input_ids, 
                                token_type_ids=None, 
                                attention_mask=b_input_mask)

        batch_preds_prob = test_output[0].detach().cpu().numpy()       
        preds.append(batch_preds_prob)
        
    result += np.vstack(preds) / KFOLDS
    
re = np.argmax(result, axis=1)

Some weights of the model checkpoint at scibert_scivocab_uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkp

load model done!
begin inference


Some weights of the model checkpoint at scibert_scivocab_uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkp

load model done!
begin inference


Some weights of the model checkpoint at scibert_scivocab_uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkp

load model done!
begin inference


Some weights of the model checkpoint at scibert_scivocab_uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkp

load model done!
begin inference


Some weights of the model checkpoint at scibert_scivocab_uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkp

load model done!
begin inference


In [ ]:
sub = pd.DataFrame()
sub['id'] = test['node idx']
sub['label'] = re
sub.to_csv('submission_retry.csv', index=False)